<a href="https://colab.research.google.com/github/ahmedhussein18102002/Grid-Domination-Pattern-Clash-Game/blob/main/AI_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##________________________ Alaa Gaber _______________________________




In [ ]:
###__________________________ Before Any Update _________________________________
#################################################################################
###########___________DON'T ADD ANY CODE HERE!!!!!_____________________##########
#################################################################################

class GridDominationGameAI:
    def __init__(self):
        self.grid = [["." for _ in range(5)] for _ in range(5)]
        self.scores = {"P1": 0, "P2": 0}
        self.current_player = "P1"
        self.pattern_scores = {
            "3-line": 5,    # Horizontal, vertical, or diagonal line of 3
            "2x2-square": 6,
            "L-shape": 7,
            "full-row-col": 10,
        }
        self.used_cells = set()  # To keep track of cells used in any shape

    def print_grid(self):
        print("  A  B  C  D  E")
        for i, row in enumerate(self.grid, 1):
            print(f"{i} {'  '.join(row)}")

    def is_valid_move(self, row, col):
        return 0 <= row < 5 and 0 <= col < 5 and self.grid[row][col] == "."

    def make_move(self, row, col):
        if not self.is_valid_move(row, col):
            return False
        self.grid[row][col] = "X" if self.current_player == "P1" else "O"
        self.update_score(row, col)
        self.current_player = "P1" if self.current_player == "P2" else "P2"
        return True

    def update_score(self, row, col):
        player_symbol = "X" if self.current_player == "P1" else "O"
        score = 0

        # Check for all patterns
        if self.check_horizontal(row, player_symbol):
            score += self.pattern_scores["3-line"]
        if self.check_vertical(col, player_symbol):
            score += self.pattern_scores["3-line"]
        if self.check_diagonals(player_symbol):
            score += self.pattern_scores["3-line"]
        if self.check_square(row, col, player_symbol):
            score += self.pattern_scores["2x2-square"]
        if self.check_l_shape(row, col, player_symbol):
            score += self.pattern_scores["L-shape"]

        if score > 0:
            print(f"{self.current_player} scored {score} points for a valid pattern!")

        self.scores[self.current_player] += score

    def check_horizontal(self, row, symbol):
        for col in range(3):
            if all(self.grid[row][col + i] == symbol for i in range(3)):
                self.used_cells.update([(row, col + i) for i in range(3)])  # Mark these cells as used
                return True
        return False

    def check_vertical(self, col, symbol):
        for row in range(3):
            if all(self.grid[row + i][col] == symbol for i in range(3)):
                self.used_cells.update([(row + i, col) for i in range(3)])  # Mark these cells as used
                return True
        return False

    def check_diagonals(self, symbol):
        # Check all possible 3-length diagonals
        for row in range(3):
            for col in range(3):
                if (self.grid[row][col] == symbol and
                    self.grid[row + 1][col + 1] == symbol and
                    self.grid[row + 2][col + 2] == symbol):  # forward diagonal
                    self.used_cells.update([(row, col), (row + 1, col + 1), (row + 2, col + 2)])  # Mark these cells as used
                    return True
                if (self.grid[row][col + 2] == symbol and
                    self.grid[row + 1][col + 1] == symbol and
                    self.grid[row + 2][col] == symbol):  # backward diagonal
                    self.used_cells.update([(row, col + 2), (row + 1, col + 1), (row + 2, col)])  # Mark these cells as used
                    return True
        return False

    def check_square(self, row, col, symbol):
        # Check if the current move is part of a 2x2 square
        for dr, dc in [(0, 0), (0, -1), (-1, 0), (-1, -1)]:
            if 0 <= row + dr < 4 and 0 <= col + dc < 4:
                if all(self.grid[row + dr + i][col + dc + j] == symbol for i in range(2) for j in range(2)):
                    self.used_cells.update([(row + dr + i, col + dc + j) for i in range(2) for j in range(2)])  # Mark as used
                    return True
        return False

    def check_l_shape(self, row, col, symbol):
        # Check all possible L-shape configurations
        l_shapes = [
            [(0, 0), (1, 0), (2, 0), (0, 1)],  # L rotated 0 degrees
            [(0, 0), (0, 1), (0, 2), (1, 0)],  # L rotated 90 degrees
            [(0, 0), (1, 0), (2, 0), (2, 1)],  # L rotated 180 degrees
            [(0, 0), (0, 1), (0, 2), (1, 2)],  # L rotated 270 degrees
        ]
        for dr, dc in [(0, 0), (-1, 0), (0, -1), (-1, -1)]:
            if 0 <= row + dr < 3 and 0 <= col + dc < 3:
                for shape in l_shapes:
                    if all(
                        0 <= row + dr + r < 5 and 0 <= col + dc + c < 5 and self.grid[row + dr + r][col + dc + c] == symbol
                        for r, c in shape
                    ):
                        self.used_cells.update([(row + dr + r, col + dc + c) for r, c in shape])  # Mark as used
                        return True
        return False

    def is_game_over(self):
        return all(cell != "." for row in self.grid for cell in row)

    def get_winner(self):
        if self.scores["P1"] > self.scores["P2"]:
            return "P1"
        elif self.scores["P1"] < self.scores["P2"]:
            return "P2"
        return "Tie"

    def evaluate(self):
        return self.scores["P1"] - self.scores["P2"]

    def minimax(self, depth, alpha, beta, maximizing_player):
        if depth == 0 or self.is_game_over():
            return self.evaluate()

        if maximizing_player:
            max_eval = float('-inf')
            for row in range(5):
                for col in range(5):
                    if self.grid[row][col] == ".":
                        self.grid[row][col] = "O"  # AI's symbol
                        eval = self.minimax(depth - 1, alpha, beta, False)
                        self.grid[row][col] = "."  # Undo move
                        max_eval = max(max_eval, eval)
                        alpha = max(alpha, eval)
                        if beta <= alpha:
                            break
            return max_eval
        else:
            min_eval = float('inf')
            for row in range(5):
                for col in range(5):
                    if self.grid[row][col] == ".":
                        self.grid[row][col] = "X"  # Player's symbol
                        eval = self.minimax(depth - 1, alpha, beta, True)
                        self.grid[row][col] = "."  # Undo move
                        min_eval = min(min_eval, eval)
                        beta = min(beta, eval)
                        if beta <= alpha:
                            break
            return min_eval

    def best_move(self):
        best_score = float('-inf')
        best_move = None
        for row in range(5):
            for col in range(5):
                if self.grid[row][col] == ".":
                    self.grid[row][col] = "O"
                    move_score = self.minimax(3, float('-inf'), float('inf'), False)
                    self.grid[row][col] = "."
                    if move_score > best_score:
                        best_score = move_score
                        best_move = (row, col)
        return best_move

    def play(self):
        print("Welcome to Grid Domination: Pattern Clash with AI!")
        while not self.is_game_over():
            self.print_grid()
            print(f"Current scores: {self.scores}")

            if self.current_player == "P1":
                move = input(f"Enter your move (row col): ").upper()
                col = move[0]  # First character is the column letter
                row = int(move[1]) - 1  # Second character is the row number (adjusted for 0 index)

                # Convert column letter (A, B, C, etc.) to index (0, 1, 2, etc.)
                col_index = ord(col) - ord('A')

                if not self.make_move(row, col_index):
                    print("Invalid move. Try again.")
                    continue
            else:
                print("AI is making its move...")
                row, col = self.best_move()
                self.make_move(row, col)
                print(f"AI plays at {row}, {col}")

        self.print_grid()
        winner = self.get_winner()
        print(f"Game Over! Winner is: {winner}")


# Run the game with AI
if __name__ == "__main__":
    game = GridDominationGameAI()
    game.play()




In [ ]:
##________________________ Ahmed Hussein ___________________________

class GridDominationGameAI:
    def __init__(self, difficulty, p1_char, p2_char):
        self.grid_size = {"easy": 4, "medium": 5, "hard": 6}[difficulty.lower()]
        self.grid = [["." for _ in range(self.grid_size)] for _ in range(self.grid_size)]
        self.scores = {"P1": 0, "P2": 0}
        self.current_player = "P1"
        self.p1_char = p1_char
        self.p2_char = p2_char
        self.pattern_scores = {
            "3-line": 5,  # Horizontal, vertical, or diagonal line of 3
            "2x2-square": 6,
            "L-shape": 7,
            "full-row-col": 10,
        }
        self.used_cells = set()  # To keep track of cells used in any shape

    def print_grid(self):
        print("   " + "  ".join(chr(ord('A') + i) for i in range(self.grid_size)))
        for i, row in enumerate(self.grid, 1):
            print(f"{i} {'  '.join(row)}")

    def is_valid_move(self, row, col):
        return 0 <= row < self.grid_size and 0 <= col < self.grid_size and self.grid[row][col] == "."

    def make_move(self, row, col):
        if not self.is_valid_move(row, col):
            return False
        self.grid[row][col] = self.p1_char if self.current_player == "P1" else self.p2_char
        self.update_score(row, col)
        self.current_player = "P1" if self.current_player == "P2" else "P2"
        return True

    def update_score(self, row, col):
        player_symbol = self.p1_char if self.current_player == "P1" else self.p2_char
        score = 0

        # Check for all patterns
        if self.check_horizontal(row, player_symbol):
            score += self.pattern_scores["3-line"]
        if self.check_vertical(col, player_symbol):
            score += self.pattern_scores["3-line"]
        if self.check_diagonals(player_symbol):
            score += self.pattern_scores["3-line"]
        if self.check_square(row, col, player_symbol):
            score += self.pattern_scores["2x2-square"]
        if self.check_l_shape(row, col, player_symbol):
            score += self.pattern_scores["L-shape"]

        if score > 0:
            print(f"{self.current_player} scored {score} points for a valid pattern!")

        self.scores[self.current_player] += score

    def check_horizontal(self, row, symbol):
        for col in range(self.grid_size - 2):
            if all(self.grid[row][col + i] == symbol for i in range(3)):
                self.used_cells.update([(row, col + i) for i in range(3)])  # Mark these cells as used
                return True
        return False

    def check_vertical(self, col, symbol):
        for row in range(self.grid_size - 2):
            if all(self.grid[row + i][col] == symbol for i in range(3)):
                self.used_cells.update([(row + i, col) for i in range(3)])  # Mark these cells as used
                return True
        return False

    def check_diagonals(self, symbol):
        for row in range(self.grid_size - 2):
            for col in range(self.grid_size - 2):
                if (self.grid[row][col] == symbol and
                    self.grid[row + 1][col + 1] == symbol and
                    self.grid[row + 2][col + 2] == symbol):
                    self.used_cells.update([(row, col), (row + 1, col + 1), (row + 2, col + 2)])  # Mark these cells as used
                    return True
                if (self.grid[row][col + 2] == symbol and
                    self.grid[row + 1][col + 1] == symbol and
                    self.grid[row + 2][col] == symbol):
                    self.used_cells.update([(row, col + 2), (row + 1, col + 1), (row + 2, col)])  # Mark these cells as used
                    return True
        return False

    def check_square(self, row, col, symbol):
        for dr, dc in [(0, 0), (0, -1), (-1, 0), (-1, -1)]:
            if 0 <= row + dr < self.grid_size - 1 and 0 <= col + dc < self.grid_size - 1:
                if all(self.grid[row + dr + i][col + dc + j] == symbol for i in range(2) for j in range(2)):
                    self.used_cells.update([(row + dr + i, col + dc + j) for i in range(2) for j in range(2)])  # Mark as used
                    return True
        return False

    def check_l_shape(self, row, col, symbol):
        l_shapes = [
            [(0, 0), (1, 0), (2, 0), (0, 1)],  # L rotated 0 degrees
            [(0, 0), (0, 1), (0, 2), (1, 0)],  # L rotated 90 degrees
            [(0, 0), (1, 0), (2, 0), (2, 1)],  # L rotated 180 degrees
            [(0, 0), (0, 1), (0, 2), (1, 2)],  # L rotated 270 degrees
        ]
        for dr, dc in [(0, 0), (-1, 0), (0, -1), (-1, -1)]:
            if 0 <= row + dr < self.grid_size - 2 and 0 <= col + dc < self.grid_size - 2:
                for shape in l_shapes:
                    if all(
                        0 <= row + dr + r < self.grid_size and
                        0 <= col + dc + c < self.grid_size and
                        self.grid[row + dr + r][col + dc + c] == symbol
                        for r, c in shape
                    ):
                        self.used_cells.update([(row + dr + r, col + dc + c) for r, c in shape])  # Mark as used
                        return True
        return False

    def is_game_over(self):
        return all(cell != "." for row in self.grid for cell in row)

    def get_winner(self):
        if self.scores["P1"] > self.scores["P2"]:
            return "P1"
        elif self.scores["P1"] < self.scores["P2"]:
            return "P2"
        return "Tie"

    def best_move(self):
        return (0, 0)  # Placeholder for AI's move logic

    def play(self):
        print("Welcome to Grid Domination: Pattern Clash with AI!")
        while not self.is_game_over():
            self.print_grid()
            print(f"Current scores: {self.scores}")
            if self.current_player == "P1":
                move = input("Enter your move (row col): ").upper()
                col = move[0]
                row = int(move[1]) - 1
                col_index = ord(col) - ord('A')
                if not self.make_move(row, col_index):
                    print("Invalid move. Try again.")
                    continue
            else:
                print("AI is making its move...")
                row, col = self.best_move()
                self.make_move(row, col)
                print(f"AI plays at {row + 1}, {chr(col + ord('A'))}")

        self.print_grid()
        winner = self.get_winner()
        print(f"Game Over! Winner is: {winner}")


# Game initialization
if __name__ == "__main__":
    difficulty = input("Choose difficulty (easy, medium, hard): ").strip().lower()
    p1_char = input("Enter character for Player 1: ").strip()
    p2_char = input("Enter character for Player 2 (AI): ").strip()
    game = GridDominationGameAI(difficulty, p1_char, p2_char)
    game.play()




In [ ]:
##________________________ Mohamed Khaled ___________________________




In [ ]:
##________________________ Ali Abd_Elmonem __________________________




In [ ]:
##________________________ Ahmed khaled _____________________________





In [ ]:
##________________________ Eslam Naser _______________________________


